# Part 3: Peer-to-peer Message Behaviour Data Analysis

---

### Install Python packages (pip only)

In [1]:
#e.g., %pip install some-package
%pip install ndlib

Note: you may need to restart the kernel to use updated packages.


### Import Python packages

In [2]:
#e.g., import some-package
import networkx as nx
import numpy as np
import pandas as pd
import datetime as dt
import operator
import random
from itertools import combinations
import matplotlib.pyplot as plt

import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import ndlib.models.CompositeModel as gc
import ndlib.models.compartments as cs
from ndlib.utils import multi_runs

---

### Task 1 of 2

Examine the file "p2p_msg_cmt224.csv" which represents messaging behaviour between users on a messaging platform. Each row has four columns, representing a single event where a person (person_a) messaged another person (person_b) on some date (date) at some time of day (time). From this, answer the following questions:

##### Q1. Build a suitable network to represent social connections based on the messaging behaviour that took place in the first 28 days. In doing so, assume that one or more messages from one person to another represents a mutual underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point). 

In [3]:
#CODE:
#read the .csv file to a pandas dataframe
df = pd.read_csv("p2p_msg_cmt224.csv")

#change the type of date from str to datetime
df['date'] = pd.to_datetime(df['date'],dayfirst=True)

#get the earlies date in the dataset and add 28 days
first_28_days = df['date'].min() + dt.timedelta(days=28)

#filter the data to create a subset
df_first_28_days = df.query("date < @first_28_days").copy()

#build an undirected graph from the data
G_1st_28_days = nx.from_pandas_edgelist(df_first_28_days,
                                        source='person_a',
                                        target='person_b',
                                        create_using=nx.Graph())

G_1st_28_days.number_of_edges(),G_1st_28_days.number_of_nodes()

(6507, 1146)

ANSWER: Based on the requirement, an undirected graph was built.
- Explanation: 
    - The data were load into a pandas dataframe as it is more convenient to include the messaging behaviour only happened in the first 28 days using filter.
    - An undirected graph was created where the source is person_a and target is person_b.
- Justification:
    - An undirected graph captures the essence of social connections inferred from messaging behaviour, irrespective of the direction of messages exchanged between individuals, which is more suitable than directed graph in this case.

##### Q2. Using the largest connected component of the network constructed in Task 1, Q1. What is the mean, median and the standard deviation of the differences between the maximum degree of separation of each individual and the average distance between the individual and all others?

In [4]:
#CODE:
#Extract the connected components
Cc = nx.connected_components(G_1st_28_days)
#Extract the largest connected component
Lcc = sorted(Cc,key=len,reverse=True)[0]
#Create a subgraph for the largest connected component in first 28 days network
G_1st_28_days_lcc = G_1st_28_days.subgraph(Lcc)

#Calculate the eccentricity => The eccentricity of a node v is the maximum distance from v to all other nodes in G.
eccentricity = nx.eccentricity(G_1st_28_days_lcc)

#Find the every possible shortest path length for every node
list_of_shortest_path_length_individual = list(nx.shortest_path_length(G_1st_28_days_lcc))

#Calculate the average shortest path length, note the denominator is the length - 1 
#because the previous result contains the node itself
average_distance_individual = [(u,(sum(v.values()) / (len(v.values())-1))) for u,v in list_of_shortest_path_length_individual]

#turn data to pandas dataframe
df_average_distance_individual = pd.DataFrame(average_distance_individual)\
.rename(columns={0:'node',1:'average_distance'})

#turn data to pandas dataframe
df_eccentricity = pd.DataFrame(eccentricity,index=['eccentricity']).T.reset_index()\
.rename(columns={"index":"node"})

#merge two dataframes
df_result = df_eccentricity.merge(df_average_distance_individual,on=['node'])

#calculate the difference
df_result['difference'] = df_result["eccentricity"] - df_result["average_distance"]

#print the results
print("The mean value is {0}, the median value is {1}, the standard deviation value is {2}".format(round(df_result['difference'].describe()['mean'],2),
round(df_result['difference'].describe()['50%'],2),
round(df_result['difference'].describe()['std'],2)))

The mean value is 2.24, the median value is 2.23, the standard deviation value is 0.4


ANSWER: The mean, median and standard deviation of the differences between eccentricity and average distance are 2.24, 2.23, and 0.4 respectively.
- Explanation:
    - The largest connected component was extracted from previous network.
    - Eccentricity was calculated for the subgraph.
    - List of shortest path length for each node was extracted, and the average distance was then computed at individual level.
    - Results were stored in pandas dataframes for calculating the mean,median and std value.
- Justification:
    - Eccentricity of a node represents the maximum degree of separation of each individual and all others, which directly answers the question.
    - Average distance is the sum of shortest_path_length divided by the count of nodes - 1 since results include node itself.

##### Q3. Build another suitable network to represent social connections based on ALL message behaviour in the dataset. In doing so, assume that one or messages from one person to another represents a MUTUAL underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point).Can the social phenomenon, ‘Triadic Closure’, be supported for the common nodes that exist in both the network created from behaviour for the first 28 days (i.e., from Task 1, Q1) and the network built from all message behaviour?

In [5]:
#CODE:
#Create an undirected network based on all message behaviour
G_all = nx.from_pandas_edgelist(df,
                                source='person_a',
                                target='person_b',
                                create_using=nx.Graph())

#Calculate the transitivity for each graph
print(("Transitivity for graph(first 28 days): {0}").format(round(nx.transitivity(G_1st_28_days),2)))
print(("Transitivity for graph(entire network with common nodes): {0}").format(round(nx.transitivity(G_all.subgraph(G_1st_28_days.nodes())),2)))

Transitivity for graph(first 28 days): 0.05
Transitivity for graph(entire network with common nodes): 0.07


ANSWER: The difference of two computed transitivity is 0.02, indicates that triadic closure can be supported for the common nodes over time.
- Explanation: 
    - A new undirected graph was created based on all message behaviour where source is person_a and target is person_b. 
    - Transitivity was calculated for two networks based on the common nodes.
- Justification: 
    - Undirected graph can represent mutual social connection as direction is ignored.
    - Transitivity measures the social phenomenon 'triadic closure'. It computes the fraction of all possible triangles presented in a graph. 
    - One may consider calculating 16 possible types of triads, but it is more relevant for directed graph.

##### Q4. What hypothetical, non-existent edges would need to be added to the network representing all message behaviour (i.e., from Task 1, Q3) such that a message could pass along a path from any node to any other? In doing so, aim to minimise the number of edges that would be needed as well as the longest shortest path in the network as a result.

In [6]:
#CODE
#copy the network
G_all_copy = G_all.copy()
#find the components need to be connected
print("Components need to be connected with the largest connected component(LCC) are:", list(nx.connected_components(G_all_copy))[1:])

#Create a largest connected component
Cc_all = nx.connected_components(G_all_copy)
Lcc_all = sorted(Cc_all,key=len,reverse=True)[0]
G_lcc_all = G_all.subgraph(Lcc_all)

#The diameter is the maximum eccentricity -> longest shortest path in the network
print("The diameter of the LCC is {0}.".format(nx.diameter(G_lcc_all)))
#The radius -> minimum eccentricity
print("The radius of the LCC is {0}.".format(nx.radius(G_lcc_all)))
#print the node with eccentricity equals to the radius
print("Candidate nodes in LCC can be used for adding edges with are {0}.".format(nx.center(G_lcc_all)))

#Add the edges for the whole network
G_all_copy.add_edges_from([(32,229),(3,10),(105,1192),(308,1797),(456,1812)])

#Compute the result
print("Is it true that the network is connected?: {0}.".format(nx.is_connected(G_all_copy)))
#The diameter is the maximum eccentricity. -> longest shortest path in the network
print("The diameter of the whole graph is {0}.".format(nx.diameter(G_all_copy)))
#The radius -> minimum eccentricity
print("The radius of the whole graph is {0}.".format(nx.radius(G_all_copy)))

Components need to be connected with the largest connected component(LCC) are: [{229, 230}, {1258, 10}, {1192, 1530}, {1797, 1798}, {1812, 1813}]
The diameter of the LCC is 8.
The radius of the LCC is 4.
Candidate nodes in LCC can be used for adding edges with are [32, 3, 105, 308, 456, 719, 1713].
Is it true that the network is connected?: True.
The diameter of the whole graph is 8.
The radius of the whole graph is 4.


ANSWER: Minimum five hypothetical edges should be added between node (32,229),(3,10),(105,1192),(308,1797),(456,1812) to make the whole network connected without increase the longest shortest path(diameter).

- Explanation:
    - 6 isolated connected components were found. Five of them should be connected to the largest connected component.
    - The diameter for the LCC was computed to understand the current maximum eccentricity.
    - The candidate nodes with eccentricity equals to the radius were found.
    - Edges were then added between the candidate nodes and nodes need to be connected.
- Justification:
    - Minimum 5 edges are required to make the graph connected in this case.
    - Nodes with eccentricity equals to the radius have the shortest path(4) to all other nodes.
    - Add edges with them will have no impact on diameter.
    - A potentially worse case is to add edges with "the periphery".

### Task 2 of 2 

Using the largest connected component of the network constructed from all data in Task 1, Q2, assume the role of an outsider with complete visibility of the network that now wishes to spread a hypothetical message such that everyone in the component would know the information it contained as quickly as possible. Assume that messages will now spread in sequential timesteps using the following mechanism. If an individual is told the message at timestep 𝑡, the individual will forward the message to all of their direct connections at timestep 𝑡+1. Individuals can therefore be told the message more than once. From this, answer the following questions:

##### Q1. If you could only select 1 individual to tell at timestep 0, what set of nodes could you select from which would result in the message being received by everyone in the fewest timesteps as possible and what would the number of timesteps be?

In [7]:
#CODE
#prepare a list of nodes that will be used in for loop
list_of_nodes = list(G_1st_28_days_lcc.nodes())
num_of_nodes = G_1st_28_days_lcc.number_of_nodes()

#create a function to build the Epidemic model
def model_build(s_node,i_size):
    #if the argument is int then convert it to list
    if type(s_node) == int:
        s_node = [s_node]
    else:
        s_node = s_node
    #Using the largest connected component
    model = ep.SIModel(G_1st_28_days_lcc)
    model_configuration = mc.Configuration()
    #set the model parameter 'beta' to 1 = (100%)
    model_configuration.add_model_parameter('beta',1)
    #set the initial configuration as per the given parameter
    model_configuration.add_model_initial_configuration("Infected",s_node)
    #set the status
    model.set_initial_status(model_configuration)
    #given the i_size parameter, set the iteration size
    iterations = model.iteration_bunch(bunch_size = i_size)
    #get the trends result
    trends = model.build_trends(iterations)
    #get the least time step to reach the whole network
    try:
        least_timestep = (trends[0]['trends']['node_count'][1].index(num_of_nodes)) + 1
    except:
        least_timestep = np.nan
    #get the result of the last iteration from the node_count result in trends
    infected_result = trends[0]['trends']['node_count'][1][i_size-1]
    return least_timestep, infected_result

#append the timestep
least_timestep_list = []

#for loop each node
for node in list_of_nodes:
    # 10 is an arbitrary iteration number, if the min is equal to 10, iteration number should be adjusted to test.
    x, _ = model_build(node,10)
    least_timestep_list.append(x)

#find the min
min_least_timestep_list = min(least_timestep_list)

#print the answer
print("least time step is {0}".format(min_least_timestep_list))
      
#create an empty list for holding the result
result_list = []
      
#iterate through all nodes and append the result to the list
for node in list_of_nodes:
    #use the min_least_timestep_list to re-run the code
    _, y = model_build(node, min_least_timestep_list)
    result_list.append(y)

#create a pandas dataframe to store the result
df_result_t2q1 = pd.DataFrame({"node_id":list_of_nodes,
                               "number_of_nodes_infected":result_list})\
.query("number_of_nodes_infected == @G_1st_28_days_lcc.number_of_nodes()")

#print the result
print(df_result_t2q1['node_id'].to_list())

least time step is 5
[32, 34, 9, 52, 58, 12, 8, 72, 73, 88, 194, 209, 252, 254, 272, 297, 308, 309, 349, 365, 368, 372, 377, 394, 409, 448, 454, 475, 481, 482, 513, 527, 542, 586, 617, 638, 640, 652, 654, 673, 679, 681, 683, 686, 687, 697, 712, 711, 713, 753, 758, 823, 834, 986]


ANSWER: The individual can be chosen from above list of nodes. The fewest timestep is 5 for the whole network to be infected for given nodes.
- Explanation:
    - SIModel was used to stimulate the spread of information.
    - During 10 iterations, the minimum timestep (5) was found.
    - The model was re-run using 5 as iteration to find all nodes that result in diffuse message in this fewest timesteps.
- Justification:
    - The model parameter 'beta' was set to 1. Because if someone was told the message, it is 100% that they have received/infected, assuming that the message platform is well functioning without loss of message during the process.
    - 10 iterations is an arbitrary iteration number and should be adjust case by case.

##### Q2. If you had to select any 5 individuals to tell at timestep 0, can the message be received by everyone in fewer timesteps than the single individual selection in Q1? In determining your answer, use one or more appropriate network connectivity measures, rather than an exhaustive search through every combination of nodes in the network.

In [8]:
#Code:
#define the model
def model_build_q2(s_node,i_size):
    #if the argument is int then convert it to list
    if type(s_node) == int:
        s_node = [s_node]
    else:
        s_node = s_node
    #Using the largest connected component
    model = ep.SIModel(G_1st_28_days_lcc)
    model_configuration = mc.Configuration()
    #set the model parameter 'beta' to 1 = (100%)
    model_configuration.add_model_parameter('beta',1)
    #set the initial configuration as per the given parameter
    model_configuration.add_model_initial_configuration("Infected",s_node)
    #set the status
    model.set_initial_status(model_configuration)
    #given the i_size parameter, set the iteration size
    iterations = model.iteration_bunch(bunch_size = i_size)
    #get the trends result
    trends = model.build_trends(iterations)
    #get the result of the last iteration from the node_count result in trends
    infected_result = trends[0]['trends']['node_count'][1][i_size-1]
    return infected_result

#how many nodes?
k = 5

def sort_result(x):
    result = sorted(
        [(node[0],node[1]) for node in dict(x).items()],
        key=operator.itemgetter(1),
        reverse=True)
    return [node for (node,value) in result[:k]]

#find top 5 nodes with highest degree
candidates_degree = sort_result(nx.degree(G_1st_28_days_lcc))

#find top 5 nodes with highest pagerank
candidates_pagerank_centrality = sort_result(nx.pagerank(G_1st_28_days_lcc))

#find top 5 nodes with highest eigenvector
candidates_eigenvector = sort_result(nx.eigenvector_centrality(G_1st_28_days_lcc))

#find top 5 nodes with highest betweenness
candidates_betweenness = sort_result(nx.betweenness_centrality(G_1st_28_days_lcc))

#find top 5 nodes with highest closeness
candidates_closeness_centrality = sort_result(nx.closeness_centrality(G_1st_28_days_lcc))

#find top 5 nodes with highest degree centrality
candidates_degree_centrality = sort_result(nx.degree_centrality(G_1st_28_days_lcc))


#combine all candiates into a set
all_candidates = set(candidates_degree + 
                     candidates_pagerank_centrality + 
                     candidates_eigenvector + 
                     candidates_betweenness + 
                     candidates_closeness_centrality + 
                     candidates_degree_centrality)

#create all possible candidates
all_possible_candidates = list(combinations(all_candidates, 5))

#for loop - can timestep further reduced to 3 iterations?
print("can timestep further reduced to 3 iterations?")
for combination in all_possible_candidates:
    if model_build_q2(combination,3) == G_1st_28_days_lcc.number_of_nodes():
        print(combination, model_build_q2(combination,3))
    else:
        continue
print("-----------------end of result------------------")        
print("can timestep further reduced to 4 iterations?")
#for loop - can timestep further reduced to 4 iterations?
for combination in all_possible_candidates:
    if model_build_q2(combination,4) == G_1st_28_days_lcc.number_of_nodes():
        print(combination, model_build_q2(combination,4))
    else:
        continue
print("-----------------end of result------------------") 

can timestep further reduced to 3 iterations?
-----------------end of result------------------
can timestep further reduced to 4 iterations?
(32, 194, 103, 9, 638) 1144
(32, 194, 41, 9, 638) 1144
(32, 194, 9, 105, 638) 1144
(32, 194, 9, 400, 638) 1144
(32, 103, 41, 9, 638) 1144
(32, 103, 9, 105, 638) 1144
(32, 103, 9, 400, 638) 1144
(32, 41, 9, 105, 638) 1144
(32, 41, 9, 400, 638) 1144
(32, 9, 105, 400, 638) 1144
(194, 103, 41, 9, 638) 1144
(194, 103, 9, 105, 638) 1144
(194, 103, 9, 400, 638) 1144
(194, 41, 9, 105, 638) 1144
(194, 41, 9, 400, 638) 1144
(194, 9, 105, 400, 638) 1144
-----------------end of result------------------


ANSWER: By choosing 5 individuals to tell at timestep 0, the timestep can be further reduced to 4 iterations.
- Explanation:
    - The model used here for stimulation was same to Q1.
    - Candidates were generated using measures(non-exhaustive) considering the different roles they play in a network.
    - All combinations include 5 individuals were generated from the candidates set.
    - Using iteration batch size(3/4), it was found that timestep can be further reduced to 4.
- Justification:
    - Nodes with high degree centrality, Pagerank centrality, eigenvector centrality, betweenness centrality, or closeness centrality can be good candidates for the analysis, as each centrality measure captures a different aspect of a node's importance within the network.